## Import the relevant package

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

C:\Users\msano\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28,28,1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

### Choose the optimizer and the loss function

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [5]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - 6ms/step - accuracy: 0.9196 - loss: 0.2706 - val_accuracy: 0.9615 - val_loss: 0.1237
Epoch 2/5
540/540 - 2s - 4ms/step - accuracy: 0.9694 - loss: 0.1006 - val_accuracy: 0.9762 - val_loss: 0.0730
Epoch 3/5
540/540 - 2s - 4ms/step - accuracy: 0.9777 - loss: 0.0689 - val_accuracy: 0.9843 - val_loss: 0.0519
Epoch 4/5
540/540 - 2s - 3ms/step - accuracy: 0.9839 - loss: 0.0506 - val_accuracy: 0.9835 - val_loss: 0.0492
Epoch 5/5
540/540 - 2s - 3ms/step - accuracy: 0.9871 - loss: 0.0411 - val_accuracy: 0.9885 - val_loss: 0.0349


## Test the model

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.9799 - loss: 0.0683
